In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim

# Load the dataset
file_path = 'earthquakes.csv'
data = pd.read_csv(file_path)

# Convert 'Time' to datetime format
data['Time'] = pd.to_datetime(data['Time'])

# Handle missing values
data.dropna(inplace=True)

# Normalize the numerical data
numerical_features = ['Magnitude', 'Depth', 'Latitude', 'Longitude']
scaler = StandardScaler()
data[numerical_features] = scaler.fit_transform(data[numerical_features])

# Define categories based on magnitude
def categorize_magnitude(magnitude):
    if magnitude <= 4.0:
        return 0  # Low magnitude
    elif magnitude <= 6.0:
        return 1  # Moderate magnitude
    else:
        return 2  # High magnitude

# Apply the function to create a new column 'Category'
data['Category'] = data['Magnitude'].apply(categorize_magnitude)


In [3]:
# Prepare the data
X = data[numerical_features]
y = data['Category']

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [4]:
# Logistic Regression
log_reg = LogisticRegression()
log_reg.fit(X_train, y_train)

# Predictions and evaluation
y_pred_log_reg = log_reg.predict(X_test)
accuracy_log_reg = accuracy_score(y_test, y_pred_log_reg)
report_log_reg = classification_report(y_test, y_pred_log_reg)

print(f'Logistic Regression Accuracy: {accuracy_log_reg}')
print(f'Logistic Regression Report:\n{report_log_reg}')


Logistic Regression Accuracy: 1.0
Logistic Regression Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      3489
           1       1.00      1.00      1.00         2

    accuracy                           1.00      3491
   macro avg       1.00      1.00      1.00      3491
weighted avg       1.00      1.00      1.00      3491



In [5]:
# Convert data to PyTorch tensors
X_tensor_train = torch.tensor(X_train, dtype=torch.float32)
y_tensor_train = torch.tensor(y_train.values, dtype=torch.long)
X_tensor_test = torch.tensor(X_test, dtype=torch.float32)
y_tensor_test = torch.tensor(y_test.values, dtype=torch.long)

# Create DataLoader
train_dataset = TensorDataset(X_tensor_train, y_tensor_train)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Define the neural network
class EarthquakeClassifier(nn.Module):
    def __init__(self):
        super(EarthquakeClassifier, self).__init__()
        self.fc1 = nn.Linear(len(numerical_features), 64)
        self.fc2 = nn.Linear(64, 3)  # 3 output classes

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Instantiate the model, define the loss function and the optimizer
model = EarthquakeClassifier()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 20
for epoch in range(num_epochs):
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}')

# Evaluation of the neural network
model.eval()
with torch.no_grad():
    test_outputs = model(X_tensor_test)
    _, test_predictions = torch.max(test_outputs, 1)

accuracy_nn = accuracy_score(y_tensor_test, test_predictions)
report_nn = classification_report(y_tensor_test, test_predictions)

print(f'Neural Network Accuracy: {accuracy_nn}')
print(f'Neural Network Report:\n{report_nn}')


ValueError: could not determine the shape of object type 'DataFrame'